In [1]:
!pip3 install transformers sagemaker --upgrade

     |████████████████████████████████| 2.9 MB 24.7 MB/s eta 0:00:01
     |████████████████████████████████| 442 kB 70.5 MB/s eta 0:00:01
     |████████████████████████████████| 52 kB 169 kB/s  eta 0:00:01
     |████████████████████████████████| 895 kB 73.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 51.5 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-2.59.4-py2.py3-none-any.whl size=623242 sha256=5581ebe78d5a1f033565a925162d6b2f5e003dffbb039f409daf3beb3b049c5a
  Stored in directory: /home/ec2-user/.cache/pip/wheels/b7/5c/a5/f504fc8a5622feb7de18eb13eccf261fc66e54f541a2e42e71
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.59.3
    Uninstalling sagemaker-2.59.3:
      Successfully uninstalled sagemaker-2.59.3


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
MODEL = 'distilbert-base-uncased-finetuned-sst-2-english'
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model.save_pretrained('my_model')
tokenizer.save_pretrained('my_model')

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

('my_model/tokenizer_config.json',
 'my_model/special_tokens_map.json',
 'my_model/vocab.txt',
 'my_model/added_tokens.json',
 'my_model/tokenizer.json')

In [3]:
!cd my_model && tar zcvf model.tar.gz * 
!mv my_model/model.tar.gz ./model.tar.gz

config.json
pytorch_model.bin
special_tokens_map.json
tokenizer_config.json
tokenizer.json
vocab.txt


In [4]:
import sagemaker
from sagemaker.s3 import S3Uploader,s3_path_join

# get the s3 bucket
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
sagemaker_session_bucket = sess.default_bucket()
# uploads a given file to S3.
upload_path = s3_path_join("s3://",sagemaker_session_bucket,"neo_test")
model_uri = S3Uploader.upload('model.tar.gz',upload_path)

In [5]:
model_uri

's3://sagemaker-us-east-1-558105141721/neo_test/model.tar.gz'

In [8]:
from sagemaker.huggingface import HuggingFaceModel

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    model_data=model_uri,
	transformers_version='4.6.1',
	pytorch_version='1.7.1',
	py_version='py36',
	role=role, 
)


In [26]:
huggingface_model.compile(
    target_instance_family="ml_inf1", # entifies the device that you want to run on, for example: ml_c5. 
    input_shape =  {'input_ids': [1,1,128], 'attention_mask': [1,1,128]},
    output_path = s3_path_join("s3://",sagemaker_session_bucket,"neo_test_output"),
    role=role,
    job_name="neo-test",
    compile_max_run=900,
    framework="pytorch",
    framework_version="1.7.1"
)

????????????????????????????????????.....................................................*

UnexpectedStatusException: Error for Compilation job neo-test: Failed. Reason: ClientError: InputConfiguration: No pth file found for PyTorch model. Please make sure the framework you select is correct.